get unread threads facebook?


In [ ]:
var importer = require('../Core');
var {
    loginFacebook,
    listFacebookThreads,
    readFacebookThread
} = importer.import([
    'decrypt password',
    'log in facebook',
    'list facebook threads',
    'messages from facebook'
], {client})

function getUnreadThreads() {
    return client
        .then(() => loginFacebook())
        .getUrl()
        .then(url => url.indexOf('https://www.facebook.com/messages/?filter=unread') === -1
             ? client.url('https://www.facebook.com/messages/?filter=unread')
             : [])
        .pause(500)
        .then(() => listFacebookThreads(false))
        .then(threads => {
            return importer.runAllPromises(threads.map(thread => resolve => {
                return readFacebookThread(thread)
                    .catch(e => console.log(e))
                    .then(r => resolve(r))
            }))
        })
        .catch(e => console.log(e))
}
module.exports = getUnreadThreads;


scan commands facebook?



In [ ]:
var importer = require('../Core');
var runSeleniumCell = importer.import('selenium cell');
var {
    filterCommand,
    storeResult
} = importer.import([
    'filter command permission',
    'store rpc result'
]);

var getUnreadThreads, sendFacebookMessage;
function scanCommandsFacebook() {
    return runSeleniumCell([
        'unread threads facebook',
        'send facebook message',
    ])
        .then(r => {
            getUnreadThreads = r.getUnreadThreads;
            sendFacebookMessage = r.sendFacebookMessage;
            return getUnreadThreads();
        })
        .then(friends => {
            const hasCommands = friends.map(f => Object.assign(f, {
                messages: f.messages
                    .filter(m => m.from !== 'Brian' && (/megamind\s*(.*)/ig).exec(m.message))
            })).filter(f => f.messages.length > 0).map(f => Object.assign(f, {
                name: f.participants[0].name.split('\n')[0],
                command: (/megamind\s*(.*)/ig).exec(f.messages[0].message)[1].trim(),
                date: f.messages[0].time,
                id: 'Facebook: ' + f.thread.replace(/^\/|\/$/ig, '').split('/').pop()
                    + f.messages[0].time
            }));
            return importer.runAllPromises(hasCommands.map(f => resolve => {
                return filterCommand(f.command, f.date, f.id, f.name)
                    .then(props => resolve(Object.assign(f, props)));
            }))
        })
        // generate and send responses
        .then(friends => {
            return importer.runAllPromises(friends.map(f => resolve => {
                return storeResult(f)
                    .then(response => sendFacebookMessage(JSON.stringify(response, null, 4) + '\n Mm\n', f.thread))
                    .catch(e => console.log(e))
                    .then(r => resolve(r))
            }))
        })
        .catch(e => console.log(e))
}
module.exports = scanCommandsFacebook;

if(typeof $$ !== 'undefined') {
    $$.async();
    scanCommandsFacebook()
        .then(r => $$.sendResult(r))
        .catch((e) => $$.sendError(e))
}


sync facebook threads?



In [ ]:
var importer = require('../Core');
var fs = require('fs');
var runSeleniumCell = importer.import('selenium cell');
var loginFacebook, listFacebookThreads, readFacebookThread;

var PROFILE_PATH = process.env.HOME || process.env.HOMEPATH || process.env.USERPROFILE;
var project = PROFILE_PATH + '/Conversations';

$$.async();
runSeleniumCell([
    'log in facebook',
    'list facebook threads',
    'messages from facebook',
])
    .then(r => ({
        loginFacebook, listFacebookThreads, readFacebookThread
    } = r).loginFacebook())
    .then(threads => {
        try { threads = JSON.parse(fs.readFileSync(project + '/facebook-threads.json')); }
        catch (e) { threads = []; }
    // TODO: compare glob/stat to see which threads need updating
        return importer.runAllPromises(threads.slice(450, 600)
            .map(t => ((resolve) => readFacebookThread(t).then((r) => resolve(r)))));
    })
    // TODO: when threads gets to 100% call listFacebookThreads()
    //.then(() => listFacebookThreads())


List Facebook threads?



In [ ]:
var fs = require('fs');

var PROFILE_PATH = process.env.HOME || process.env.HOMEPATH || process.env.USERPROFILE;
var project = PROFILE_PATH + '/Conversations';

function getFacebookThreads(threads) {
    return client
        .getAllUntil('//*[contains(@class, "scrollable")]',
                     '//*[contains(@data-href, "messages")]/@data-href',
                     threads,
                     (a, b) => a === b,
                     i => i < 20)
};

function listArchivedFacebook(threads) {
    return client
        .then(url => url.indexOf('/messages/archived') == -1
            ? client.url('https://www.facebook.com/messages/archived')
                .pause(1000)
                .click('.scrollable a[href="#"]')
            : Promise.resolve([]))
        .pause(2000)
        .then(() => getFacebookThreads(threads))
}

var threads, results;
function listFacebookThreads(threads, archived = false) {
    try {
        if(threads !== false) {
            threads = JSON.parse(fs.readFileSync(project + '/facebook-threads.json'));
        } else {
            threads = [];
        }
    }
    catch (e) {
        threads = [];
    }
    return client.getUrl()
        .then(url => url.indexOf('/messages/t') == -1
            ? client
                .click('[data-tooltip-content="Messages"]')
                .click('a[href*="/messages/t"]')
            : Promise.resolve([]))
        .pause(2000)
        .then(() => getFacebookThreads(threads))
        .then(r => threads = r)
        .then(() => archived ? listArchivedFacebook(threads) : threads)
        // list all message threads
        .then(r => {
            threads = r;
            fs.writeFileSync(
                project + '/facebook-threads.json',
                JSON.stringify(r, null, 4));
            return threads;
        })
        .catch(e => console.log(e))
};
module.exports = listFacebookThreads;
listFacebookThreads;



Get messages from facebook?



In [ ]:
var importer = require('../Core');
var chrono = require('chrono-node');
var glob = require('glob');
var fs = require('fs');
var readFacebookProfileInfo = importer.import('scrape facebook profile', {client})

var PROFILE_PATH = process.env.HOME || process.env.HOMEPATH || process.env.USERPROFILE;
var project = PROFILE_PATH + '/Conversations';

function switchToParticipantThread(i) {
    return client
        .pause(1000)
        .click('//*[contains(@class, "scrollable")]//h4[contains(., "People")]/parent::*//li[' + i + ']')
        .pause(1000)
        .then(() => readFacebookProfileInfo())
        .catch(e => console.log(e))
};

function getThreadParticipants(thread) {
    return client
        .execute(() => {
            return document.evaluate(
                'count(//*[contains(@class, "scrollable")]//h4[contains(., "People")]/parent::*//li)',
                document, null,
                XPathResult.NUMBER_TYPE, null).numberValue;
        })
        .then(r => {
            var result = [];
            for (var i = 1; i < Math.min(3, r.value); i++) {
                result[result.length] = (i =>
                    resolve => {
                        var profile = {};
                        switchToParticipantThread(i)
                            .then(r => profile = r)
                            .url(thread)
                            .pause(3000)
                            .catch(e => console.log(e))
                            .then(() => resolve(profile))
                    })(i);
            }
            if (r.value === 0) {
                result[result.length] = (resolve => {
                    var profile = {};
                    readFacebookProfileInfo()
                        .then(r => profile = r)
                        .url(thread)
                        .pause(3000)
                        .catch(e => console.log(e))
                        .then(() => resolve(profile))
                })
            }
            return importer.runAllPromises(result).catch(e => console.log(e));
        })
        .catch(e => console.log(e))
};

function readFacebookMessages(messages, i = 0) {
    const offset = (new Date()).getTimezoneOffset() / 60;
    var browserOffset;
    // TODO: add check for needing to go to LinkedIn
    // TODO: add check for needing to login
    return client
        // TODO: move this to utility?
        .execute(() => (new Date()).getTimezoneOffset())
        .then(o => browserOffset = o.value)
        .getAllUntil(
            '//*[contains(@role, "main")]//*[contains(@class, "scrollable")][contains(.,"Messages")]',
            '//*[contains(@role, "main")]//*[contains(@class, "scrollable")][contains(.,"Messages")]//h4//text()',
            [],
            (a, b) => a + '' === b + '',
            i => i < 10,
            true /* scroll up, not down */
        )
        .getAllXPath([
            '//*[contains(@role, "main")]//*[contains(@class, "scrollable")][contains(.,"Messages")]//h4',
            {
                time: './/text()',
                messages: [
                    './following-sibling::div[count(./preceding-sibling::h4)=$i+1]',
                    {
                        from: './/h5//text()',
                        message: './/*[not(name()="H5")]/text()'
                    }
                ]
            }
        ])
        .then(messages => {
            return messages.reduce((acc, m) => {
                // TODO: this should get the timezone inside the container using client.execute() and append:
                var results = new chrono.parse(m.time); // Create new ParsedResult Object
                results[0].start.assign('timezoneOffset', browserOffset * 60); // Change value in ParsedComponents Object 'start'
                var newTime = results[0].start.date(); // lazy fix
                // facebook only displays the time if the message is sent that day,
                //   this causes a problem if the timezones in the brwoser are different than the timezone here in Node.
                //   We have to reverse the effect of the timezone, check if the day is incremented and increment the newTime accordingly.
                /*
                const localTime = new Date();
                const offsetTime = new Date();
                offsetTime.setHours(offsetTime.getHours() - (browserOffset - offset));
                if(newTime.getDate() === localTime.getDate() && newTime.getDate() + 1 === offsetTime.getDate()) {
                    // the current time adjusted for GMT has incremented one day
                    newTime.setDate(newTime.getDate() + 1);
                    console.log(newTime);
                }
                */
                
                m.messages.forEach((r, i) => {
                    const newMessage = {
                        time: newTime,
                        from: r.from,
                        message: typeof r.message === 'string' ? r.message : r.message.join('\n')
                    };
                    if(i === 0 && newMessage.message === newMessage.from) {
                        return;
                    }
                    acc.push(newMessage);
                })
                return acc;
            }, []);
        })
        .catch(e => console.log(e))
};

function readFacebookThread(thread) {
    var threadId = thread.replace(/^\/|\/$/ig, '').split('/').pop()
        .replace(/[^a-z0-9]/ig, '_');
    var file = glob.sync('**/' + threadId + '-*.json', {cwd: project})[0];
    var messages;
    try {
        messages = JSON.parse(fs.readFileSync(file))
    }
    catch (e) {
        messages = []
    }
    var participants;
    return client.getUrl().then(url => url.indexOf(thread) === -1
        ? client.url(thread)
        : client)
        // get participants from topcard
        .pause(1000)
        .then(() => getThreadParticipants(thread))
        .then(p => (participants = p))

        // TODO: save to contacts
        .then(() => readFacebookMessages(messages || []))
        .then(messages => ({
            thread: thread,
            participants: participants || [],
            messages: messages
        }))
        .then(t => {
            var filename = project + '/'
                + threadId
                + '-' + t.participants
                    .map(p => p.name).join('')
                    .replace(/[^a-z0-9]/ig, '_') + '.json';
            fs.writeFileSync(filename, JSON.stringify(t, null, 4));
            return t;
        })
        .catch(e => console.log(e))
};
module.exports = readFacebookThread;


In [3]:
var importer = require('../Core');
var runSeleniumCell = importer.import('run selenium cell');

$$.async();
runSeleniumCell('messages from facebook')
    .then(func => func('https://www.facebook.com/messages/t/jaimeneufer'))
    .then(r => $$.sendResult(r))
    .catch(e => $$.sendError(e))


compiling /Users/briancullinan/jupytangular2/Selenium/webdriver.ipynb[0]
compiling /Users/briancullinan/jupytangular2/Selenium/webdriver.ipynb[1]
compiling /Users/briancullinan/jupytangular2/Selenium/webdriver.ipynb[2]
compiling /Users/briancullinan/jupytangular2/Selenium/webdriver.ipynb[4]
compiling /Users/briancullinan/jupytangular2/Selenium/webdriver.ipynb[5]
compiling /Users/briancullinan/jupytangular2/Selenium/webdriver.ipynb[3]
init - locking
init - lock
using existing session 0 - ddded66b2095959fa624974f75a41a1a
update - locking
update ddded66b2095959fa624974f75a41a1a
init - unlocking
update - lock
update ddded66b2095959fa624974f75a41a1a
update - unlocking
init - unlock
scanning on
update - unlock
update - locking
update - lock
update ddded66b2095959fa624974f75a41a1a
update - unlocking
update - unlock
[ 'ddded66b2095959fa624974f75a41a1a' ]
update - locking
update - lock
update ddded66b2095959fa624974f75a41a1a
update - unlocking
update - unlock
{ sessionId: 'ddded66b2095959fa6249

send facebook message?



In [ ]:
function sendFacebookMessage(message, thread) {
    var result;
    if(typeof thread !== 'undefined') {
        result = client.getUrl()
            .then(url => url.indexOf(thread) > -1
                  ? []
                  : client.url(thread));
    } else {
        result = client;
    }
    const parts = message.split('\n');
    return result
        .click('//*[contains(@aria-label, "Type a message")]')
        .keys('Control')
        .keys('a')
        .keys('NULL')
        .keys('Delete')
        .then(() => importer.runAllPromises(parts.map((t, i) => resolve => {
            if(i === parts.length - 1) {
                return client.keys(t).then(() => resolve());
            }
            return client
                .keys(t)
                .keys('Shift')
                .keys('Enter')
                .keys('NULL')
                .catch(e => console.log(e))
                .then(() => resolve());
        })))
        .keys('\uE007')
        .catch(e => console.log(e))
}
module.exports = sendFacebookMessage;
